In [ ]:
%load_ext autoreload

In [47]:
import sys

import altair as alt
import polars as pl

sys.path.append("../")
%autoreload
from altair_utils import color_line_muni_bart_caltrain
from utils import line_plot_normalized

In [5]:
# BART has a weird drop in 2023, so we're not using the monthly data from MTC directly
# vrh = (
#     pl.read_csv(  # vehicle revenue hours
#         r"Q:\Data\PeerAgencyDashboards\MTC-NTD\Daily VRH Line_data.csv",
#         schema_overrides={"Average Daily Vehicle Revenue Hours": float},
#     )
#     .with_columns((pl.col("Month of Date") + " 1").str.to_date("%B %Y %d"))
#     .rename({"Common.Agency.Name": "agency", "Month of Date": "date"})
#     .with_columns(year=pl.col("date").dt.year())
# )
# vrm = (
#     pl.read_csv(  # vehicle revenue miles
#         r"Q:\Data\PeerAgencyDashboards\MTC-NTD\Daily VRM Line_data.csv",
#         schema_overrides={
#             "Avg. Average Daily Vehicle Revenue Miles": float,
#             "Avg. Monthly Vehicle Revenue Miles": float,
#         },
#     )
#     .with_columns((pl.col("Month of Date")).str.to_date("%m/%d/%Y"))
#     .rename(
#         {
#             "Common.Agency.Name": "agency",
#             "Month of Date": "date",
#             "Avg. Average Daily Vehicle Revenue Miles": "Average Daily Vehicle Revenue Miles",
#         }
#     )
#     .with_columns(year=pl.col("date").dt.year())
# )

vrh = (
    pl.read_csv(  # vehicle revenue hours
        r"Q:\Model Research\downtown_today\transit-NTD-VehicleRevenueHours.csv",
        # schema_overrides={"Average Daily Vehicle Revenue Hours": float},
    )
).unpivot(
    index="year",
    on=["BART", "Caltrain", "Muni"],
    variable_name="operator",
    value_name="annual vehicle revenue hours (VRH)",
)

In [ ]:
(
    line_plot_normalized(
        vrh,
        x="year",
        y="annual vehicle revenue hours (VRH)",
        color=alt.Color("operator").scale(
            domain=["Muni", "BART", "Caltrain"], range=color_line_muni_bart_caltrain
        ),
        norm_x_value=2018,
    )[1]
    | line_plot_normalized(
        vrh,
        x="year",
        y="annual vehicle revenue hours (VRH)",
        color=alt.Color("operator").scale(
            domain=["Muni", "BART", "Caltrain"], range=color_line_muni_bart_caltrain
        ),
        norm_x_value=2019,
    )[1]
)


In [ ]:
vrh_normalized, chart = line_plot_normalized(
    vrh.filter(pl.col("year") > 2018),
    x="year",
    y="annual vehicle revenue hours (VRH)",
    color=alt.Color("operator").scale(
        domain=["Muni", "BART", "Caltrain"], range=color_line_muni_bart_caltrain
    ),
    norm_x_value=2019,
    tooltip=[
        "year",
        "operator",
        alt.Tooltip("annual vehicle revenue hours (VRH)", title="annual VRH"),
        alt.Tooltip(
            r"annual vehicle revenue hours (VRH) (% of 2019 value)",
            title=r"annual VRH (% of 2019 value)",
        ),
    ],
)
vrh_normalized.write_csv("../output/data/transit-supply-vrh.csv")
chart.save("../output/Links/transit-supply-vrh.png")
chart.properties(width="container").save("../output/Links/transit-supply-vrh.html")
chart